# <center>CSC 328 Practical Question 1</center>
## <center>Wakhisi Kevin Wasike</center>
## <center>P15/1928/2020</center>


Data for this practical was gotten from [UCI Machine Learning Repository: Online Retail Data Set](https:\\archive.ics.uci.edu\\ml\\datasets\\online+retail). This is what has been used to create the star, galaxy and snowflake schemas. The csv file has the following columns:

1. invoice\_no
2. stock\_code
3. description
4. quantity
5. invoice\_date
6. unit\_price
7. customer\_id
8. country

The tool that has been employed in this practical is [Azure Data Studio](https://learn.microsoft.com/en-us/sql/azure-data-studio/what-is-azure-data-studio?view=sql-server-ver16). It offers features such as SQL code editor, Intellisense, Jupyter Notebook integration, and the really neat thing about this is that, not only can it run python notebooks, but also SQL notebooks, as you can see in this document. Thejupyter notebook has then been exported to this pdf.

Azure Data Studio offers a modern editor experience with IntelliSense, code snippets, source control integration, and an integrated terminal. It's engineered with the data platform user in mind, with built-in charting of query result sets and customizable dashboards.

## 1\. <u>STAR SCHEMA</u>

![star](/home/waks/Repos/public/business-intelligence-and_analytics/star.png)

## Preliminaries

In [1]:
USE online_retail_star;

DROP TABLE IF EXISTS fact_sales;

DROP TABLE IF EXISTS dim_product;

DROP TABLE IF EXISTS dim_customer;

DROP TABLE IF EXISTS dim_date;

DROP TABLE IF EXISTS dim_country;

DROP TABLE IF EXISTS online_retail;

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:10.052

## Load the Dataset

In [2]:
CREATE TABLE
    online_retail (
        invoice_no VARCHAR(100),
        stock_code VARCHAR(255),
        description VARCHAR(255),
        quantity INT,
        invoice_date DATETIME,
        unit_price DECIMAL(8, 2),
        customer_id VARCHAR(100),
        country VARCHAR(100)
    );

LOAD DATA
    INFILE '/var/lib/mysql-files/online_retail.csv' INTO
TABLE
    online_retail FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED BY '\n' IGNORE 1 LINES(
        invoice_no,
        stock_code,
        description,
        quantity,
        @invoice_date,
        unit_price,
        customer_id,
        country
    )
SET
    invoice_date = STR_TO_DATE(
        @invoice_date,
        '%d/%m/%Y %H:%i'
    );

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:40.369

## Create and populate Dimension Tables

### dim\_product

In [3]:
CREATE TABLE
    dim_product (
        stock_code VARCHAR(255) PRIMARY KEY,
        description VARCHAR(255)
    );

INSERT INTO
    dim_product (stock_code, description)
SELECT
    DISTINCT stock_code,
    description
FROM online_retail;

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:18.904

### dim\_customer

In [4]:
CREATE TABLE
    dim_customer (
        customer_id INT AUTO_INCREMENT PRIMARY KEY,
        customer_name VARCHAR(255),
        country VARCHAR(100)
    );

INSERT INTO
    dim_customer (customer_name, country)
SELECT
    DISTINCT customer_id,
    country
FROM online_retail;

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:08.239

### dim\_date

In [5]:
CREATE TABLE
    dim_date (
        invoice_date DATETIME PRIMARY KEY,
        day INT,
        month INT,
        year INT
    );

INSERT INTO
    dim_date (invoice_date, day, month, year)
SELECT
    DISTINCT invoice_date,
    DAY(invoice_date),
    MONTH(invoice_date),
    YEAR(invoice_date)
FROM online_retail;

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:14.304

### dim\_country

In [6]:
CREATE TABLE
    dim_country (
        country VARCHAR(100) PRIMARY KEY
    );

INSERT INTO
    dim_country (country)
SELECT DISTINCT country
FROM online_retail;

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:05.468

## Create and populate fact table

### fact\_sales

In [7]:
CREATE TABLE
    fact_sales (
        fact_id INT AUTO_INCREMENT PRIMARY KEY,
        stock_code VARCHAR(255),
        customer_id INT,
        invoice_date DATETIME,
        country VARCHAR(100),
        total_price DECIMAL(10, 2),
        FOREIGN KEY (stock_code) REFERENCES dim_product(stock_code),
        FOREIGN KEY (customer_id) REFERENCES dim_customer(customer_id),
        FOREIGN KEY (invoice_date) REFERENCES dim_date(invoice_date),
        FOREIGN KEY (country) REFERENCES dim_country(country)
    );

INSERT INTO
    fact_sales (
        stock_code,
        customer_id,
        invoice_date,
        country,
        total_price
    )
SELECT
    ORT.stock_code,
    DC.customer_id,
    ORT.invoice_date,
    ORT.country, (ORT.quantity * ORT.unit_price) AS total_price
FROM online_retail ORT
    LEFT JOIN dim_product DP ON ORT.stock_code = DP.stock_code
    LEFT JOIN dim_customer DC ON ORT.customer_id = DC.customer_id
    LEFT JOIN dim_date DD ON ORT.invoice_date = DD.invoice_date
    LEFT JOIN dim_country DCO ON ORT.country = DCO.country;

Commands completed successfully

Commands completed successfully

Total execution time: 00:03:49.886

# 2. <u>GALAXY SCHEMA</u>

![galaxy](/home/waks/Repos/public/business-intelligence-and_analytics/galaxy.png)

## Preliminaries

In [8]:
USE online_retail_galaxy;

DROP TABLE IF EXISTS fact_sales;

DROP TABLE IF EXISTS fact_returns;

DROP TABLE IF EXISTS dim_product;

DROP TABLE IF EXISTS dim_customer;

DROP TABLE IF EXISTS dim_date;

DROP TABLE IF EXISTS dim_country;

DROP TABLE IF EXISTS online_retail;

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:13.212

## Load the Dataset

In [9]:
CREATE TABLE
    online_retail (
        invoice_no VARCHAR(100),
        stock_code VARCHAR(255),
        description VARCHAR(255),
        quantity INT,
        invoice_date DATETIME,
        unit_price DECIMAL(8, 2),
        customer_id VARCHAR(100),
        country VARCHAR(100)
    );

LOAD DATA
    INFILE '/var/lib/mysql-files/online_retail.csv' INTO
TABLE
    online_retail FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED BY '\n' IGNORE 1 LINES(
        invoice_no,
        stock_code,
        description,
        quantity,
        @invoice_date,
        unit_price,
        customer_id,
        country
    )
SET
    invoice_date = STR_TO_DATE(
        @invoice_date,
        '%d/%m/%Y %H:%i'
    );

Commands completed successfully

Commands completed successfully

Total execution time: 00:02:32.568

## Create and populate Dimension Tables

### dim\_product

In [10]:
CREATE TABLE
    dim_product (
        stock_code VARCHAR(255) PRIMARY KEY,
        description VARCHAR(255),
        category VARCHAR(255) DEFAULT 'Uncategorized',
        supplier VARCHAR(255) DEFAULT 'Unknown'
    );

INSERT INTO
    dim_product (
        stock_code,
        description,
        category,
        supplier
    )
SELECT
    DISTINCT stock_code,
    description,
    'Uncategorized',
    'Unknown'
FROM online_retail;

Commands completed successfully

Total execution time: 00:00:04.504

### dim\_customer

In [11]:
CREATE TABLE
    dim_customer (
        customer_id INT AUTO_INCREMENT PRIMARY KEY,
        customer_name VARCHAR(255),
        country VARCHAR(100),
        segment VARCHAR(255) DEFAULT 'Unspecified',
        loyalty_status VARCHAR(255) DEFAULT 'Not Enrolled'
    );

INSERT INTO
    dim_customer (
        customer_name,
        country,
        segment,
        loyalty_status
    )
SELECT
    DISTINCT customer_id,
    country,
    'Unspecified',
    'Not Enrolled'
FROM online_retail;

Commands completed successfully

Total execution time: 00:00:03.441

### dim\_date

In [12]:
CREATE TABLE
    dim_date (
        invoice_date DATETIME PRIMARY KEY,
        day INT,
        month INT,
        year INT
    );

INSERT INTO
    dim_date (invoice_date, day, month, year)
SELECT
    DISTINCT invoice_date,
    DAY(invoice_date),
    MONTH(invoice_date),
    YEAR(invoice_date)
FROM online_retail;

Commands completed successfully

Total execution time: 00:00:04.106

### dim\_country

In [13]:
CREATE TABLE
    dim_country (
        country VARCHAR(100) PRIMARY KEY
    );

INSERT INTO
    dim_country (country)
SELECT DISTINCT country
FROM online_retail;

Commands completed successfully

Total execution time: 00:00:03.647

## Create and populate fact tables

### fact\_sales

In [14]:
CREATE TABLE
    fact_sales (
        fact_sales_id INT AUTO_INCREMENT PRIMARY KEY,
        stock_code VARCHAR(255),
        customer_id INT,
        invoice_date DATETIME,
        country VARCHAR(100),
        total_price DECIMAL(10, 2),
        FOREIGN KEY (stock_code) REFERENCES dim_product(stock_code),
        FOREIGN KEY (customer_id) REFERENCES dim_customer(customer_id),
        FOREIGN KEY (invoice_date) REFERENCES dim_date(invoice_date),
        FOREIGN KEY (country) REFERENCES dim_country(country)
    );

INSERT INTO
    fact_sales (
        stock_code,
        customer_id,
        invoice_date,
        country,
        total_price
    )
SELECT
    ORT.stock_code,
    DC.customer_id,
    ORT.invoice_date,
    ORT.country, (ORT.quantity * ORT.unit_price) AS total_price
FROM online_retail ORT
    JOIN dim_product DP ON ORT.stock_code = DP.stock_code
    JOIN dim_customer DC ON ORT.customer_id = DC.customer_id
    JOIN dim_date DD ON ORT.invoice_date = DD.invoice_date
    JOIN dim_country DCO ON ORT.country = DCO.country;

Commands completed successfully

Total execution time: 00:00:07.170

### fact\_returns

In [15]:
CREATE TABLE
    fact_returns (
        fact_returns_id INT AUTO_INCREMENT PRIMARY KEY,
        stock_code VARCHAR(255),
        customer_id INT,
        return_date DATETIME,
        country VARCHAR(100),
        return_quantity INT,
        return_price DECIMAL(10, 2),
        FOREIGN KEY (stock_code) REFERENCES dim_product(stock_code),
        FOREIGN KEY (customer_id) REFERENCES dim_customer(customer_id),
        FOREIGN KEY (return_date) REFERENCES dim_date(invoice_date),
        FOREIGN KEY (country) REFERENCES dim_country(country)
    );

    INSERT INTO
    fact_returns (
        stock_code,
        customer_id,
        return_date,
        country,
        return_quantity,
        return_price
    )
SELECT
    ORT.stock_code,
    DC.customer_id,
    ORT.invoice_date,
    ORT.country,
    ORT.quantity, (ORT.quantity * ORT.unit_price) AS return_price
FROM online_retail ORT
    JOIN dim_product DP ON ORT.stock_code = DP.stock_code
    JOIN dim_customer DC ON ORT.customer_id = DC.customer_id
    JOIN dim_date DD ON ORT.invoice_date = DD.invoice_date
    JOIN dim_country DCO ON ORT.country = DCO.country
WHERE ORT.quantity < 0;

Commands completed successfully

Total execution time: 00:00:07.844

## 3\. <u>SNOWFLAKE SCHEMA</u>

![snowflake](/home/waks/Repos/public/business-intelligence-and_analytics/snowflake.png)

## Preliminaries

In [22]:
USE online_retail_snowflake;

DROP TABLE IF EXISTS fact_sales;

DROP TABLE IF EXISTS fact_returns;

DROP TABLE IF EXISTS dim_product;

DROP TABLE IF EXISTS dim_customer;

DROP TABLE IF EXISTS dim_date;

DROP TABLE IF EXISTS dim_country;

DROP TABLE IF EXISTS dim_category;

DROP TABLE IF EXISTS dim_supplier;

DROP TABLE IF EXISTS online_retail;

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

...

Commands completed successfully

Commands completed successfully

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:23.073

## Load the Dataset

In [23]:
CREATE TABLE
    online_retail (
        invoice_no VARCHAR(100),
        stock_code VARCHAR(255),
        description VARCHAR(255),
        quantity INT,
        invoice_date DATETIME,
        unit_price DECIMAL(8, 2),
        customer_id VARCHAR(100),
        country VARCHAR(100)
    );

LOAD DATA
    INFILE '/var/lib/mysql-files/online_retail.csv' INTO
TABLE
    online_retail FIELDS TERMINATED BY ',' ENCLOSED BY '"' LINES TERMINATED BY '\n' IGNORE 1 LINES(
        invoice_no,
        stock_code,
        description,
        quantity,
        @invoice_date,
        unit_price,
        customer_id,
        country
    )
SET
    invoice_date = STR_TO_DATE(
        @invoice_date,
        '%d/%m/%Y %H:%i'
    );

Commands completed successfully

Commands completed successfully

Total execution time: 00:02:31.466

## Create and populate Dimension Tables

### dim\_product

In [24]:
CREATE TABLE
    dim_product (
        product_id INT AUTO_INCREMENT PRIMARY KEY,
        stock_code VARCHAR(255),
        description VARCHAR(255),
        category VARCHAR(255) DEFAULT 'Uncategorized',
        supplier VARCHAR(255) DEFAULT 'Unknown'
    );

INSERT INTO
    dim_product (
        stock_code,
        description,
        category,
        supplier
    )
SELECT
    DISTINCT stock_code,
    description,
    'Uncategorized',
    'Unknown'
FROM online_retail;

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:10.707

### dim\_customer

In [25]:
CREATE TABLE
    dim_customer (
        customer_id INT AUTO_INCREMENT PRIMARY KEY,
        customer_name VARCHAR(255),
        country VARCHAR(100),
        segment VARCHAR(255) DEFAULT 'Unspecified',
        loyalty_status VARCHAR(255) DEFAULT 'Not Enrolled'
    );

INSERT INTO
    dim_customer (
        customer_name,
        country,
        segment,
        loyalty_status
    )
SELECT
    DISTINCT customer_id,
    country,
    'Unspecified',
    'Not Enrolled'
FROM online_retail;

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:09.374

### dim\_date

In [26]:
CREATE TABLE
    dim_date (
        date_id INT AUTO_INCREMENT PRIMARY KEY,
        invoice_date DATETIME,
        day INT,
        month INT,
        year INT
    );

INSERT INTO
    dim_date (invoice_date, day, month, year)
SELECT
    DISTINCT invoice_date,
    DAY(invoice_date),
    MONTH(invoice_date),
    YEAR(invoice_date)
FROM online_retail;

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:15.342

### dim\_country

In [27]:
CREATE TABLE
    dim_country (
        country_id INT AUTO_INCREMENT PRIMARY KEY,
        country VARCHAR(100)
    );

INSERT INTO
    dim_country (country)
SELECT DISTINCT country
FROM online_retail;


Commands completed successfully

Commands completed successfully

Total execution time: 00:00:06.116

### dim\_category

In [28]:
CREATE TABLE
    dim_category (
        category_id INT AUTO_INCREMENT PRIMARY KEY,
        category_name VARCHAR(255)
    );

INSERT INTO
    dim_category (category_name)
SELECT DISTINCT category
FROM dim_product;

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:03.494

### dim\_supplier

In [29]:
CREATE TABLE
    dim_supplier (
        supplier_id INT AUTO_INCREMENT PRIMARY KEY,
        supplier_name VARCHAR(255)
    );

INSERT INTO
    dim_supplier (supplier_name)
SELECT DISTINCT supplier
FROM dim_product;


Commands completed successfully

Commands completed successfully

Total execution time: 00:00:04.308

## Create and populate fact tables

### fact\_sales

In [30]:
CREATE TABLE
    fact_sales (
        sales_id INT AUTO_INCREMENT PRIMARY KEY,
        product_id INT,
        customer_id INT,
        date_id INT,
        country_id INT,
        quantity INT,
        unit_price DECIMAL(8, 2),
        total_price DECIMAL(10, 2),
        FOREIGN KEY (product_id) REFERENCES dim_product(product_id),
        FOREIGN KEY (customer_id) REFERENCES dim_customer(customer_id),
        FOREIGN KEY (date_id) REFERENCES dim_date(date_id),
        FOREIGN KEY (country_id) REFERENCES dim_country(country_id)
    );

INSERT INTO
    fact_sales (
        product_id,
        customer_id,
        date_id,
        country_id,
        quantity,
        unit_price,
        total_price
    )
SELECT
    DP.product_id,
    DC.customer_id,
    DD.date_id,
    DCO.country_id,
    ORT.quantity,
    ORT.unit_price, (ORT.quantity * ORT.unit_price) AS total_price
FROM online_retail ORT
    JOIN dim_product DP ON ORT.stock_code = DP.stock_code
    JOIN dim_customer DC ON ORT.customer_id = DC.customer_id
    JOIN dim_date DD ON ORT.invoice_date = DD.invoice_date
    JOIN dim_country DCO ON ORT.country = DCO.country;

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:06.718

### fact\_returns

In [31]:
# 
CSC 328 Practical Question 1
## 
Wakhisi Kevin Wasike
## 
P15/1928/2020

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:06.511